In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import fisher_exact

In [3]:
#load main table
copd_tb_f = '../results/copd_table_post_filtering.tsv'
tb = pd.read_csv(copd_tb_f, sep='\t', index_col='eid')

#add assessment centre date
ac_f = 'ukb_general_info.tsv'
ac = pd.read_csv(ac_f, sep='\t', usecols=['eid', 'ac1_date'], index_col='eid')
tb = tb.join(ac)

#change dtypes of date columns
date_cols = [col for col in tb.columns if col.endswith('date')]
dtypes = {col: np.datetime64 for col in date_cols}
tb = tb.astype(dtypes)

#load spirmoetry data
fev_fvc_f = 'ukb_fev_fvc.tsv'
fev_fvc = pd.read_csv(fev_fvc_f, sep='\t', index_col='eid')

#calculate FEV1/FVC ratio
fev_fvc['FEV1_FVC'] = fev_fvc['FEV1_best'] / fev_fvc['FVC_best']

#remove samples not meeting spirometry qc
fev_fvc = fev_fvc[fev_fvc['spiro_reproducible'] == 1].dropna()

#merge tables and remove samples with no spirmetry
tb = fev_fvc.join(tb)

#exclude date cols from dropna
col_subset = [col for col in tb.columns if col not in date_cols]
tb.dropna(subset=col_subset, inplace=True)

tb

,FEV1_best,FVC_best,spiro_reproducible,FEV1_pred,FVC_pred,FEV1_pc_pred,FVC_pc_pred,FEV1_FVC,sex,height,...,death_date,event,hypertension_comb,smoking,smoking_status_0,smoking_status_1,smoking_status_2,TDI_binned,yrs_duration,ac1_date
eid,,,,,,,,,,,,,,,,,,,,,
4519111,3.52,4.30,1.0,3.102346,4.030128,113.462515,106.696361,0.818605,1.0,171.0,...,NaT,0.0,1.0,1.0,0.0,1.0,0.0,"(-6.258, -2.806]",20.0,2010-03-03
4809135,2.24,3.37,1.0,3.597970,4.424156,62.257337,76.172719,0.664688,1.0,168.0,...,NaT,0.0,0.0,2.0,0.0,0.0,1.0,"(0.646, 4.097]",20.0,2008-05-21
4941063,2.60,3.40,1.0,2.011663,2.650145,129.246323,128.294850,0.764706,0.0,153.0,...,NaT,0.0,1.0,0.0,1.0,0.0,0.0,"(-2.806, 0.646]",20.0,2009-12-16
4175004,2.53,3.62,1.0,2.934887,3.791244,86.204345,95.483164,0.698895,1.0,166.0,...,NaT,0.0,1.0,0.0,1.0,0.0,0.0,"(-2.806, 0.646]",20.0,2007-12-13
1867608,4.47,5.73,1.0,4.266738,5.364514,104.763861,106.813031,0.780105,1.0,185.0,...,NaT,0.0,1.0,0.0,1.0,0.0,0.0,"(-2.806, 0.646]",20.0,2010-04-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304133,3.10,3.83,1.0,2.867387,3.674634,108.112367,104.228079,0.809399,0.0,168.0,...,NaT,0.0,1.0,1.0,0.0,1.0,0.0,"(-2.806, 0.646]",20.0,2009-04-17
1876884,2.81,3.40,1.0,2.546838,3.327971,110.332902,102.164340,0.826471,0.0,166.0,...,NaT,0.0,1.0,1.0,0.0,1.0,0.0,"(-6.258, -2.806]",20.0,2008-04-24
1149407,2.10,2.49,1.0,2.297695,2.957077,91.395928,84.204762,0.843373,0.0,154.0,...,NaT,0.0,0.0,0.0,1.0,0.0,0.0,"(-2.806, 0.646]",20.0,2009-03-13


In [5]:
#define a new binary outcome based on pre-AC COPD diagnosis AND spirometry at the ac
pre_ac_copd = tb['event_date'] < tb['ac1_date']
ac_obs_spiro = tb['FEV1_FVC'] < 0.7
tb['copd_sens'] = np.where(pre_ac_copd & ac_obs_spiro, 1, 0)
tb['copd_sens'].value_counts()

0    266771
1      1778
Name: copd_sens, dtype: int64

In [7]:
#fit an age/sex adjusted logistic regression
age_sex_form = 'copd_sens ~ chd + age_at_ac1 + sex'
age_sex_mod = smf.logit(formula=age_sex_form, data=tb)
age_sex_res = age_sex_mod.fit()
print(age_sex_res.summary())
print(f'exp(coef) CHD = {np.exp(age_sex_res.params.chd)}')
print(f'CHD P value: {age_sex_res.pvalues.chd:.2e}')

Optimization terminated successfully.
         Current function value: 0.037950
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:              copd_sens   No. Observations:               268549
Model:                          Logit   Df Residuals:                   268545
Method:                           MLE   Df Model:                            3
Date:                Mon, 03 Oct 2022   Pseudo R-squ.:                 0.04694
Time:                        09:29:44   Log-Likelihood:                -10191.
converged:                       True   LL-Null:                       -10693.
Covariance Type:            nonrobust   LLR p-value:                2.538e-217
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.5068      0.245    -47.044      0.000     -11.986     -11.027
chd            0.7160      0

In [8]:
tb['ethnicity'].value_counts()

1.0    268549
Name: ethnicity, dtype: int64

In [9]:
#fit a fully-adjusted logistic regression
fa_form = ('copd_sens ~ chd + age_at_ac1 + sex + bmi + smoking_status_1 + ' 
          'smoking_status_2 + asthma + hypertension_comb + Townsend_DI')
fa_mod = smf.logit(formula=fa_form, data=tb)
fa_res = fa_mod.fit()
print(fa_res.summary())
print(f'exp(coef) CHD = {np.exp(fa_res.params.chd)}')
print(f'CHD P value: {fa_res.pvalues.chd:.2e}')

Optimization terminated successfully.
         Current function value: 0.031567
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:              copd_sens   No. Observations:               268549
Model:                          Logit   Df Residuals:                   268539
Method:                           MLE   Df Model:                            9
Date:                Mon, 03 Oct 2022   Pseudo R-squ.:                  0.2072
Time:                        09:30:03   Log-Likelihood:                -8477.4
converged:                       True   LL-Null:                       -10693.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           -13.2387      0.303    -43.706      0.000     -13.832     -12.645
chd    

In [10]:
def get_n_pc(data, group_mask, column, val, dp=1):
    x = data[group_mask][column]
    n = (x == val).sum()
    group_size = len(x)
    pc = 100*n / group_size
    return f'{n:,} ({pc:.1f})'

def get_conf_int_str(res):
    conf_int = np.exp(res.conf_int().loc['chd'].to_numpy())
    return f'({conf_int[0]:.2f}, {conf_int[1]:.2f})'

groups = ['CHD', 'Control']
masks = [tb['chd'] == 1, tb['chd'] == 0]

rows = [
    ['No. at Risk'] + [f'{mask.sum():,}' for mask in masks],
    ['COPD Cases, n (%)'] + [get_n_pc(tb, mask, 'copd_sens', 1) for mask in masks],
    ['Age/Sex Adjusted:'] + ['' for group in groups],
    ['OR', f'{np.exp(age_sex_res.params.chd):.2f}', '-'],
    ['95% CI', get_conf_int_str(age_sex_res), '-'],
    ['P-Value', f'{age_sex_res.pvalues.chd:.2e}', '-'],
    ['Fully Adjusted:'] + ['' for group in groups],
    ['OR', f'{np.exp(fa_res.params.chd):.2f}', '-'],
    ['95% CI', get_conf_int_str(fa_res), '-'],
    ['P-Value', f'{fa_res.pvalues.chd:.2e}', '-'],
    
]

rows = np.array(rows)
final = pd.DataFrame(rows[:, 1:], index=rows[:, 0], columns=groups)
final

,CHD,Control
No. at Risk,"1,784","266,765"
"COPD Cases, n (%)",27 (1.5),"1,751 (0.7)"
Age/Sex Adjusted:,,
OR,2.05,-
95% CI,"(1.39, 3.01)",-
P-Value,2.63e-04,-
Fully Adjusted:,,
OR,1.81,-
95% CI,"(1.22, 2.68)",-
P-Value,3.33e-03,-
